<h1><center>PISAP: Python Interactive Sparse Astronomical Data Analysis Packages</center></h1>
<h2><center>DIctionary learning tutorial</center></h2>
<div style="text-align: center">Credit: </div>

Pisap is a Python package related to sparsity and its application in
astronomical or mediacal data analysis. This package propose sparse denosing methods reusable in various contexts.
For more information please visit the project page on github: https://github.com/neurospin/pisap.<br><br>

<h3>First check</h3>

In order to test if the 'pisap' package is installed on your machine, you can check the package version:

In [1]:
import pisap
print pisap.__version__

0.0.0


<h2>Decomposition / recomposition of images in a learned dictionary</h2>

The package provides a flexible implementation of a dictionary learning method.

In [2]:
import numpy as np
from scipy.io import loadmat
import scipy.fftpack as pfft
import matplotlib.pyplot as plt
%matplotlib inline

from pisap.data import get_sample_data
from pisap.base.utils import convert_mask_to_locations
from pisap.numerics.noise import add_noise
from pisap.numerics.reconstruct import sparse_rec_fista
from pisap.numerics.gradient import Grad2DSynthesis
from pisap.numerics.fourier import FFT
from pisap.numerics.cost import snr, ssim
from pisap.numerics.linear import DictionaryLearningWavelet

In [3]:
__disp_patches__ = False

First, we load the atoms of our dictionary from a '.npy' file.

In [4]:
dico = np.load("data/dico_patches_size100x49_30subjects_squareimgs_156x156.npy")
d1, d2 = dico.shape
atoms = np.zeros((int(np.sqrt(d2)), int(np.sqrt(d2)), d1))
for idx, atom in enumerate(dico):
    atoms[:, :, idx] = atom.reshape(int(np.sqrt(d2)), int(np.sqrt(d2)))
del dico

In [5]:
if __disp_patches__:
    fig, axes = plt.subplots(figsize=(10, 10), nrows=10, ncols=10)
    i = 0
    for row in axes:
        for ax in row:
            ax.axis('off')
            ax.matshow(atoms[:, :, i], cmap='gray')
            i += 1
    plt.suptitle('Dictionary', fontsize=22)
    plt.show()

Then, we define our dictionary.

In [6]:
img = np.load("data/masked_normalized_img_testingset_156x156.npy")
#dico = DictionaryLearningWavelet(atoms, img.shape, n_jobs_transform=-1)

Finally, we decompose and re-compose a brain image.

In [7]:
#coef = dico.op(img)
#recons_img = dico.adj_op(coef)

In [8]:
#print("Original image shape: {0}".format(img.shape))
#print("Coefficients shape: {0}".format(coef.shape))
#print("Reconsturcted image shape: {0}".format(recons_img.shape))

In [9]:
#fig, axes = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
#axes[0].axis('off')
#axes[0].matshow(img, cmap='gray')
#axes[0].set_title("Ref image")
#axes[1].axis('off')
#axes[1].matshow(recons_img, cmap='gray')
#axes[1].set_title("Decomposed/recomposed image")
#plt.show()

<h2> CS reconstruction with a learned dictionary</h2>

The package provides a flexible implementation of a dictionary learning representation for the reconstruction functions.  
First, we load the input k-space and the under-sampling scheme.

In [10]:
mask = loadmat("data/scheme_256_R5_power1_fullCenter.mat")['sigma']
c = int(mask.shape[0]/2)
d = 156
d_2 = int(d/2)
mask = mask[c-d_2:c+d_2, c-d_2:c+d_2]
loc = convert_mask_to_locations(pfft.ifftshift(mask))
kspace = pfft.ifftshift(mask) * pfft.ifft2(img)
kspace = add_noise(kspace, sigma=0.1)

In [11]:
metrics = {'snr':{'metric':snr,
                  'mapping': {'x_new': 'test', 'y_new':None},
                  'cst_kwargs':{'ref':img},
                  'early_stopping': False,
                   },
            'ssim':{'metric':ssim,
                  'mapping': {'x_new': 'test', 'y_new':None},
                  'cst_kwargs':{'ref':img},
                  'early_stopping': True,
                   },
           }
params = {
    'data':kspace,
    'gradient_cls':Grad2DSynthesis,
    'gradient_kwargs':{"ft_cls": {FFT: {"samples_locations": loc,
                                        "img_size": img.shape[0]}}},
    'linear_cls':DictionaryLearningWavelet,
    'linear_kwargs':{"atoms": atoms, "image_size": img.shape, "n_jobs_transform": -1},
    'max_nb_of_iter':100,
    'mu':2.0e-2,
    'metrics':metrics,
    'verbose':1,
}

x, y, saved_metrics = sparse_rec_fista(**params)

Starting FISTA reconstruction algorithm.


/home/hcherkaoui/COSMIC/src/pisap/pisap/numerics/linear.py:203: ComplexWarning: Casting complex values to real discards the imaginary part
  return self._op_real_data(data.astype('float64'))


866.01668741


/home/hcherkaoui/COSMIC/src/pisap/pisap/numerics/linear.py:257: ComplexWarning: Casting complex values to real discards the imaginary part
  img = reconstruct_from_patches_2d(patches.astype('float64'), self.image_size)


0.902522121221
0.0


/home/hcherkaoui/COSMIC/src/pisap/pisap/numerics/gradient.py:90: RuntimeWarning: invalid value encountered in divide
  x_new = self.MtMX(x_old) / np.linalg.norm(x_old)
/home/hcherkaoui/COSMIC/src/pisap/pisap/numerics/gradient.py:91: RuntimeWarning: invalid value encountered in absolute
  if(np.abs(np.linalg.norm(x_new) - np.linalg.norm(x_old)) < tolerance):


nan


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
plt.figure()
plt.imshow(mask, cmap='gray')
plt.title("Mask")

plt.figure()
plt.imshow(np.abs(pfft.ifft2(kspace), interpolation="nearest", cmap="gist_stern")
plt.colorbar()
plt.title("Dirty image")

plt.figure()
plt.imshow(np.abs(x.data), interpolation="nearest", cmap="gist_stern")
plt.colorbar()
plt.title("Analytic sparse reconstruction via Condat-Vu method")

metric = saved_metrics['snr']
fig = plt.figure()
plt.grid()
plt.plot(metric['time'], metric['values'])
plt.xlabel("time (s)")
plt.ylabel("SNR")
plt.title("Evo. SNR per time")

metric = saved_metrics['nrmse']
fig = plt.figure()
plt.grid()
plt.plot(metric['time'], metric['values'])
plt.xlabel("time (s)")
plt.ylabel("NRMSE")
plt.title("Evo. NRMSE per time")

plt.show()